#WELCOME
In this Assignment, you'll use **Classifier** to predict the output. You are allowed to write/edit between the
```# START CODE HERE``` & ```# END CODE HERE``` only.<br>
Don't edit the test cells otherwise you'd be failed in the assignment.

Upload the ```utils.zip``` file before running the 1st cell.
[Demo](https://drive.google.com/file/d/1ebkQR6ikjGottmxa5KL83ZWdjTLvRxbm/view?usp=share_link)

**BEST OF LUCK**

**RUN THIS CODE TO UNIZP THE REQUIRED FILES**

In [ ]:
!unzip -j /content/utils.zip -d.

Archive:  /content/utils.zip
  inflating: ./weather_classification_data.csv  
  inflating: ./data_scaled_labeled_check.csv  


# Dataset Loading and Pre-Processing

In [ ]:
import pandas as pd
data = pd.read_csv("/content/weather_classification_data.csv")

## Data Info
```data.info``` gives info about data-types and number of null objects.  
There are no null data, no need to remove any.

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13200 entries, 0 to 13199
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Temperature           13200 non-null  float64
 1   Humidity              13200 non-null  int64  
 2   Wind Speed            13200 non-null  float64
 3   Precipitation (%)     13200 non-null  float64
 4   Cloud Cover           13200 non-null  object 
 5   Atmospheric Pressure  13200 non-null  float64
 6   UV Index              13200 non-null  int64  
 7   Season                13200 non-null  object 
 8   Visibility (km)       13200 non-null  float64
 9   Location              13200 non-null  object 
 10  Weather Type          13200 non-null  object 
dtypes: float64(5), int64(2), object(4)
memory usage: 1.1+ MB


## Data Shuffling
Data Shuffling removes any bias present and creates uniform distribution.

In [ ]:
from sklearn.utils import shuffle
data = shuffle(data)
data.head()

,Temperature,Humidity,Wind Speed,Precipitation (%),Cloud Cover,Atmospheric Pressure,UV Index,Season,Visibility (km),Location,Weather Type
7481,-2.0,99,17.0,96.0,overcast,990.96,0,Winter,4.5,inland,Snowy
6886,24.0,60,7.0,21.0,partly cloudy,1007.33,2,Spring,8.0,inland,Cloudy
6723,18.0,70,12.5,77.0,partly cloudy,1010.19,1,Summer,1.0,inland,Rainy
3305,23.0,49,0.5,1.0,clear,1018.88,7,Autumn,9.0,coastal,Sunny
10335,62.0,100,20.5,99.0,clear,1014.46,0,Summer,6.5,mountain,Sunny


## Scaling and Labeling
Machine can't understand string so all the object datatypes need to be LABELED with integers.<br><br>
Why SCALING?<br>
You might have noticed one column contains data with large values (of order 1000) whereas some column contain small value data (of order 10). As a result, large value columns get more power/importance than others. To remove this disparity we scaled them to same order/values.

**CODE**

We need to find the columns with object datatypes.

HINT: Use ```data.select_dtypes().columns.tolist()```<br>Reference: [w3schools](https://www.w3schools.com/python/pandas/ref_df_select_dtypes.asp#:~:text=The%20select_dtypes()%20method%20returns,the%20specified%20dtype(s).&text=Note%3A%20You%20must%20specify%20at,you%20will%20get%20an%20error.)

In [ ]:
# START CODE HERE
object_columns = data.select_dtypes(include=['object']).columns.tolist()
non_object_columns = data.select_dtypes(exclude=['object']).columns.tolist()
# END CODE HERE
print(f"Object Columns: {object_columns}\nNon Object Columns: {non_object_columns}")

Object Columns: ['Cloud Cover', 'Season', 'Location', 'Weather Type']
Non Object Columns: ['Temperature', 'Humidity', 'Wind Speed', 'Precipitation (%)', 'Atmospheric Pressure', 'UV Index', 'Visibility (km)']


**TEST** (DON'T EDIT THIS CELL)

In [ ]:
# DON'T EDIT THIS CELL
if (object_columns == ['Cloud Cover', 'Season', 'Location', 'Weather Type'] and non_object_columns == ['Temperature', 'Humidity', 'Wind Speed', 'Precipitation (%)', 'Atmospheric Pressure', 'UV Index', 'Visibility (km)']):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed


**CODE**

We need to convert the object-type columns data into integral labels and scale the non-object-type columns data.

HINT: Use ```ColumnTransformer()```<br>Reference: [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), [ChatGPT](https://chatgpt.com/c/7f3c1b02-5da0-4c34-bdb2-d999dccb9c2f)

In [ ]:
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

# START CODE HERE
column_transformer = ColumnTransformer(
    transformers=[
        ('cat', OrdinalEncoder(), object_columns),
        ('num', StandardScaler(), non_object_columns)
    ]
)

# Fit and transform the data
data_scaled_labeled = column_transformer.fit_transform(data)
# END CODE HERE

all_columns = object_columns + non_object_columns

# Create the new DataFrame
data_scaled_labeled = pd.DataFrame(data_scaled_labeled, columns=all_columns)


**TEST** (DON'T EDIT THIS CELL)

In [ ]:
# DON'T EDIT THIS CELL

data_scaled_labeled_check = pd.read_csv("/content/data_scaled_labeled_check.csv")
import numpy as np
data_scaled_labeled_values = np.sort(data_scaled_labeled.values,axis=0)
data_scaled_labeled_check_values = np.sort(data_scaled_labeled_check.values,axis=0)
if np.allclose(data_scaled_labeled_values, data_scaled_labeled_check_values, equal_nan=True):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed


## Data Splitting

**CODE**

We need to drop the Weather Type column from data_scaled_labeled to get the X and pick Weather Type column to get Y

Use 90-10 split of data, i.e. 90% for training, 10% for testing

In [ ]:
# START CODE HERE
X = data_scaled_labeled.drop(columns=['Weather Type'])
y = data_scaled_labeled['Weather Type']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.90, random_state=10)
# END CODE HERE

print(f"Length of X_train: {len(X_train)}, Length of X_test: {len(X_test)}")
print(f"Length of y_train: {len(y_train)}, Length of y_test: {len(y_test)}")
X_train.head()


Length of X_train: 11880, Length of X_test: 1320
Length of y_train: 11880, Length of y_test: 1320


,Cloud Cover,Season,Location,Temperature,Humidity,Wind Speed,Precipitation (%),Atmospheric Pressure,UV Index,Visibility (km)
7017,3.0,1.0,0.0,-0.525006,-0.332327,-0.192836,0.386773,0.011347,-0.779410,-0.730539
12278,2.0,2.0,2.0,-0.294931,-0.629452,0.024290,-0.301903,0.080706,-0.001493,0.752538
2859,2.0,3.0,2.0,-1.157711,-0.381848,-1.423215,0.605898,-0.536260,-1.038715,-1.027154
5495,2.0,3.0,1.0,-1.560341,0.311445,0.530916,0.856326,-0.173070,-0.779410,-0.433923
8469,2.0,3.0,1.0,-1.330267,-0.084722,-0.337587,0.511987,-0.559111,-0.779410,-1.027154


**TEST** (DON'T EDIT THIS CELL)

In [ ]:
# DON'T EDIT THIS CELL

if (len(X_train)==11880 and len(X_test)==1320):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

if (len(y_train)==11880 and len(y_test)==1320):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed
Test Passed


In [ ]:
X_train.head()

,Cloud Cover,Season,Location,Temperature,Humidity,Wind Speed,Precipitation (%),Atmospheric Pressure,UV Index,Visibility (km)
7017,3.0,1.0,0.0,-0.525006,-0.332327,-0.192836,0.386773,0.011347,-0.779410,-0.730539
12278,2.0,2.0,2.0,-0.294931,-0.629452,0.024290,-0.301903,0.080706,-0.001493,0.752538
2859,2.0,3.0,2.0,-1.157711,-0.381848,-1.423215,0.605898,-0.536260,-1.038715,-1.027154
5495,2.0,3.0,1.0,-1.560341,0.311445,0.530916,0.856326,-0.173070,-0.779410,-0.433923
8469,2.0,3.0,1.0,-1.330267,-0.084722,-0.337587,0.511987,-0.559111,-0.779410,-1.027154


# Training and Testing
Use any suitable classifier to fit the training data and check the accuracy on test data

Don't forget to import that library

**CODE**

ACCURACY should be more than 90%,

1. Fit the model to X_train and y_train
2. Find the model score on X_test and y_test

In [ ]:
# START CODE HERE
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=10)
model.fit(X_train, y_train)
score = model.score(X_test, y_test)
# END CODE HERE

print("Accuracy:", score * 100)

Accuracy: 91.51515151515152


**TEST** (DON'T EDIT THIS CELL)

In [ ]:
# DON'T EDIT THIS CELL

if (score>0.90):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed


# Predicting Output (Here, Weather Type) (Optional)

In [ ]:
input = X_test.head()
input

,Cloud Cover,Season,Location,Temperature,Humidity,Wind Speed,Precipitation (%),Atmospheric Pressure,UV Index,Visibility (km)
11923,0.0,2.0,1.0,0.107699,-0.233285,-0.337587,-1.491435,0.500619,1.554340,0.010999
2331,3.0,3.0,2.0,-1.272748,-0.381848,-0.048086,0.825022,-0.448621,-1.038715,-0.433923
217,3.0,0.0,1.0,-0.064857,0.410487,0.530916,-0.301903,-0.005320,-0.779410,0.455922
1760,2.0,3.0,1.0,-1.502823,0.311445,0.386166,1.263271,-0.655621,-1.038715,-0.582231
3963,2.0,2.0,1.0,0.395293,0.113362,0.386166,1.325878,2.146667,-0.001493,3.718690


In [ ]:
y = model.predict(input)
y

array([3., 2., 0., 2., 0.])

Here, the outputs are in numbers. We can perform inverse column transfer for getiing the string values.
